<a href="https://colab.research.google.com/github/michaelfpalmer/BinaryProportionSampleSizes/blob/main/Sample_Size_Estimation_and_3d_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
The goal of this is to show that an approximation for sample size works very well for binary proportion metrics (like click-through-rate) and under standard assumptions of alpha=.05 and beta=.2) and incidence, MDE are sufficiently small. The level of sufficiency works well for most experiments in practice.

I'll show that total sample size needed is approximately 10 times pi divided by (MDE squared times incidence).

# Gut check

Given some common inputs for an experiment, we want to determine if the estimate is reasonable.

The inputs and estimate when incidence rate is 1%, MDE is 5%, beta = .2, alpha = .05, and ratio between groups is 1.

In [10]:
import math
prop1 = .01
MDE = .05
prop2 = prop1*(1+MDE)
beta = .2
alph = .05
rati = 1
estimate = 10*math.pi*(1/prop1)*(1/MDE**2)
print(estimate)

1256637.061435917


Calculate sample size for this example, compare to estimate, and calculate percent error.

In [11]:
import statsmodels.stats.api as sms

es = sms.proportion_effectsize(prop1, prop2, method='normal')
n = sms.NormalIndPower().solve_power(es, power=1-beta, alpha=alph, ratio=rati)
N = n + n*rati
error = (N - estimate)
error_fraction = error/N
print(N,estimate,error_fraction)

1273830.9752444508 1256637.061435917 0.013497798485575665


An error of 1.35%.

# Graphical Exploration

Ultimately we'd like to find the limitations of an estimate. To take the analysis further, let's look at a plot of what the sample size surface looks like across a collection of data points.

Constants

In [12]:
import numpy as np
pow = .8
alph = .05
rati = 1

Generate Grid

In [ ]:
#Example
#np.logspace(-3,np.log10(.5),30)

In [13]:
MDEs = np.logspace(-3,np.log10(.5),40)
props = np.logspace(-3,np.log10(.5),40)
MDEslin = np.linspace(.001,.5,40)
propslin = np.linspace(.001,.5,40)

In [ ]:
#idea
#def gen_grid(min_val,max_val,gridtype,gridpts)

Show difference between linear grid and log grid

In [14]:
MDEs, MDEslin

(array([0.001     , 0.00117275, 0.00137534, 0.00161292, 0.00189155,
        0.00221831, 0.00260151, 0.00305092, 0.00357795, 0.00419604,
        0.00492089, 0.00577096, 0.00676787, 0.00793701, 0.0093081 ,
        0.01091605, 0.01280176, 0.01501323, 0.01760672, 0.02064823,
        0.02421515, 0.02839825, 0.03330396, 0.03905712, 0.04580413,
        0.05371666, 0.06299605, 0.07387844, 0.08664072, 0.10160765,
        0.11916008, 0.13974463, 0.16388511, 0.19219578, 0.22539704,
        0.26433372, 0.3099966 , 0.3635476 , 0.42634939, 0.5       ]),
 array([0.001     , 0.01379487, 0.02658974, 0.03938462, 0.05217949,
        0.06497436, 0.07776923, 0.0905641 , 0.10335897, 0.11615385,
        0.12894872, 0.14174359, 0.15453846, 0.16733333, 0.18012821,
        0.19292308, 0.20571795, 0.21851282, 0.23130769, 0.24410256,
        0.25689744, 0.26969231, 0.28248718, 0.29528205, 0.30807692,
        0.32087179, 0.33366667, 0.34646154, 0.35925641, 0.37205128,
        0.38484615, 0.39764103, 0.4104359 , 0.

Get sample size for each grid point. We'll generate two sets of grids, one log-scaled about the origin to focus on some critical behavior--this will become clear soon--and another linear scaled to make sure we don't miss any global behavior.

In [15]:
triplets_log = []
for MDE in MDEs:
  for prop in props:
    prop2 = prop*(1+MDE)
    es = sms.proportion_effectsize(prop, prop2, method='normal')
    n = sms.NormalIndPower().solve_power(es, power=pow, alpha=alph, ratio=rati)
    N = n + rati*n
    triplet = ((MDE,prop,N))
    triplets_log.append(triplet)
triplets_log_array = np.array(triplets_log)
#triplets_log_array.shape

In [16]:
triplets_linear = []
for MDE in MDEslin:
  for prop in propslin:
    prop2 = prop*(1+MDE)
    es = sms.proportion_effectsize(prop, prop2, method='normal')
    n = sms.NormalIndPower().solve_power(es, power=pow, alpha=alph, ratio=rati)
    N = n + rati*n
    triplet = ((MDE,prop,N))
    triplets_linear.append(triplet)
triplets_linear_array = np.array(triplets_linear)
#triplets_linear_array.shape

# Graph from one reference frame
First let's get a look at what the plot looks like and make sure if matches some intuition.

Later on we'll plot the error.

In [17]:
import plotly.graph_objects as go

fig0 = go.Figure(data=[go.Mesh3d(
    x=triplets_log_array[:,0], #MDEs
    y=triplets_log_array[:,1], #props
    z=triplets_log_array[:,2], #sample size
    opacity=0.5,
    intensity=np.log10((triplets_log_array[:,2])), #sample size same as z above
    #color='rgba(244,22,100,0.6)'
)])
fig0.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[0,.5],),
        yaxis = dict(nticks=4, range=[0,.5],),
        zaxis = dict(nticks=4, range=[0,np.power(10,10)],),
        xaxis_title='MDE',
        yaxis_title='Incidence',
        zaxis_title='Sample Size',
         camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.25)
    )
        ),

    width=700,
    margin=dict(r=20, l=10, b=10, t=10))


fig0.show()

In [19]:
fig = go.Figure(data=[go.Mesh3d(
    x=triplets_linear_array[:,0], #MDEs
    y=triplets_linear_array[:,1], #props
    z=triplets_linear_array[:,2], #sample size
    opacity=0.5,
    intensity=np.log10((triplets_linear_array[:,2])), #sample size same as z above
    #color='rgba(244,22,100,0.6)'
)])
fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[0,.5],),
        yaxis = dict(nticks=4, range=[0,.5],),
        zaxis = dict(nticks=4, range=[0,np.power(10,10)],),
        xaxis_title='MDE',
        yaxis_title='Incidence',
        zaxis_title='Sample Size',
         camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.25)
    )
        ),

    width=700,
    margin=dict(r=20, l=10, b=10, t=10))


fig.show()

#First comments
So, we can see that the plot "blows up" at the origin. A closer look shows that sample size blows up faster with respect to MDE than it does with respect to incidence rate.

Let's look at a couple cross sections.

In [20]:
#get cross section where y = props = .001
Cross_Section_props001 = triplets_log_array[triplets_log_array[:,1]==.001]
MDE_Cross_Section_props001 = Cross_Section_props001[:,0]
z_Cross_Section_props001 = Cross_Section_props001[:,2]

In [45]:
MDE_Cross_Section_props001, z_Cross_Section_props001

(array([0.001     , 0.00117275, 0.00137534, 0.00161292, 0.00189155,
        0.00221831, 0.00260151, 0.00305092, 0.00357795, 0.00419604,
        0.00492089, 0.00577096, 0.00676787, 0.00793701, 0.0093081 ,
        0.01091605, 0.01280176, 0.01501323, 0.01760672, 0.02064823,
        0.02421515, 0.02839825, 0.03330396, 0.03905712, 0.04580413,
        0.05371666, 0.06299605, 0.07387844, 0.08664072, 0.10160765,
        0.11916008, 0.13974463, 0.16388511, 0.19219578, 0.22539704,
        0.26433372, 0.3099966 , 0.3635476 , 0.42634939, 0.5       ]),
 array([3.13797110e+10, 2.28180046e+10, 1.65925392e+10, 1.20657859e+10,
        8.77419456e+09, 6.38071455e+09, 4.64027348e+09, 3.37467646e+09,
        2.45435551e+09, 1.78509956e+09, 1.29840575e+09, 9.44463940e+08,
        6.87055644e+08, 4.99845080e+08, 3.63682349e+08, 2.64642562e+08,
        1.92600068e+08, 1.40191764e+08, 1.02063297e+08, 7.43209660e+07,
        5.41331888e+07, 3.94407167e+07, 2.87459226e+07, 2.09595844e+07,
        1.52894743e+07

In [39]:
Cross_Section_MDEs001 = triplets_log_array[triplets_log_array[:,0]==.001]
props_Cross_Section_MDEs001 = Cross_Section_MDEs001[:,1]
z_Cross_Section_MDEs001 = Cross_Section_MDEs001[:,2]

In [46]:
fig2 = go.Figure(data=[go.Scatter(
    x=MDE_Cross_Section_props001, #MDEs
    y=z_Cross_Section_props001, #samplesizes
    mode='lines'
)])
fig2.update_layout(
        xaxis = dict(nticks=4, range=[0.001,.01],),
        yaxis = dict(nticks=4, range=[0,5*np.power(10,10)],),
        xaxis_title='MDE',
        yaxis_title='Sample Size'
        )
fig2.show()

In [47]:
fig3 = go.Figure(data=[go.Scatter(
    x=props_Cross_Section_MDEs001, #MDEs
    y=z_Cross_Section_MDEs001, #samplesizes
    mode='lines'
)])
fig3.update_layout(
        xaxis = dict(nticks=4, range=[0.001,.01],),
        yaxis = dict(nticks=4, range=[0,5*np.power(10,10)],),
        xaxis_title='Incidence',
        yaxis_title='Sample Size'
        )
fig3.show()

## Which blows up faster?
Notice that Sample size blows up faster with respect to MDE than it does with respect to incidence.

This gives some evidence that our formula is on track. Since there is a 1/MDE^2 term, we would expect it to blow up faster than the 1/incidence term!

#Adding more texture
These graphs don't show the minutiae of what's going on. Are there any bumps or is there some structure? Is it really as monotonic as it looks? Is it convex (i.e. does the shape "hold water"?)

In [ ]:
t = np.linspace(0, 10, 50)
#x, y, z = np.cos(t), np.sin(t), t

fig= go.Figure(go.Mesh3d(x=triplets_log_array[:,0], #MDEs
    y=triplets_log_array[:,1], #props
    #z=triplets_actual_array[:,2], #sample size, y=y, z=z, mode='markers'
    z = triplets_log_array[:,2],
    intensity=np.log10(triplets_log_array[:,2])
                         ))

x_eye = -1.25
y_eye = 2
z_eye = 0.5

fig.update_layout(
         title='Sample Size Surface',
         width=600,
         height=600,
    scene=dict(
        xaxis_title='MDE',
        yaxis_title='Incidence',
        zaxis_title='Sample Size',
        zaxis=dict(dtick=1,type='log')),
         scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),

         updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=0.8,
                  xanchor='left',
                  yanchor='bottom',
                  pad=dict(t=45, r=10),
                  buttons=[dict(label='Play',
                                 method='animate',
                                 args=[None, dict(frame=dict(duration=60, redraw=True),
                                                             transition=dict(duration=0),
                                                             fromcurrent=True,
                                                             mode='immediate'
                                                            )]
                                            )
                                      ]
                              )
                        ]
)


def rotate_z(x, y, z, theta):
    w = x+1j*y
    return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

frames=[]
for t in np.arange(0, math.pi*2, 0.1):
    xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
    frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
fig.frames=frames

fig.show()

##Comment
From the plot and animation we can see that the plot looks convex.

# What is the best fitting curve?
Let's investigate two ways. First using all a collection of data points equally spaced. If everything works right, our estimate for the coefficient should be about 31.4.

In [55]:
from scipy.optimize import curve_fit

The assumption here is we have a function of the form a/(MDE^2*incidence). We want to find the optimal value of a which minimizes MSE over a domain of interest.

In [57]:
def objective(X,a):
    x, y = X
    return a/(x**2*y)

In [65]:
x=triplets_linear_array[:,0]
y=triplets_linear_array[:,1]
z=triplets_linear_array[:,2]
popt, pcov= curve_fit(objective, (x,y),z)
#x.shape, y.shape, z.shape
print(popt, 10*math.pi, abs(popt - 10*math.pi)/popt) #coefficient and percent error

[31.36992929] 31.41592653589793 [0.00146628]


In [66]:
x=triplets_log_array[:,0]
y=triplets_log_array[:,1]
z=triplets_log_array[:,2]
popt, pcov= curve_fit(objective, (x,y),z)
#x.shape, y.shape, z.shape
print(popt, 10*math.pi, abs(popt - 10*math.pi)/popt) #coefficient and percent error

[31.35676965] 31.41592653589793 [0.00188657]


The error on the coefficient is less than 1%!

#Appendix, old code, and examples

In [68]:
'''
# Generate random 3D data points
x = np.random.random(100)
y = np.random.random(100)
z = np.sin(x * y) + np.random.normal(0, 0.1, size=100)
data = np.array([x, y, z]).T

# Define mathematical function for curve fitting
def func(xy, a, b, c, d, e, f):
    x, y = xy
    return a + b*x + c*y + d*x**2 + e*y**2 + f*x*y

# Perform curve fitting
popt, pcov = curve_fit(func, (x, y), z)
'''

In [ ]:
#x.shape

(100,)

In [ ]:
'''import plotly.graph_objects as go
import pandas as pd

fig0 = go.Figure(data=[go.Mesh3d(
    x=triplets_actual_array[:,0], #MDEs
    y=triplets_actual_array[:,1], #props
    z=triplets_actual_array[:,2], #sample size
    opacity=0.5,
    intensity=np.log(triplets_actual_array[:,2]), #sample size same as z above
    #color='rgba(244,22,100,0.6)'
)])
fig0.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[0,.5],),
        yaxis = dict(nticks=4, range=[0,.5],),
        zaxis = dict(nticks=4, range=[0,1000000000],),
         camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=-1.25, z=1.25)
    )
        ),

    width=700,
    margin=dict(r=20, l=10, b=10, t=10))


fig0.show()
'''

"import plotly.graph_objects as go\nimport pandas as pd\n\nfig0 = go.Figure(data=[go.Mesh3d(\n    x=triplets_actual_array[:,0], #MDEs\n    y=triplets_actual_array[:,1], #props\n    z=triplets_actual_array[:,2], #sample size\n    opacity=0.5,\n    intensity=np.log(triplets_actual_array[:,2]), #sample size same as z above\n    #color='rgba(244,22,100,0.6)'\n)])\nfig0.update_layout(\n    scene = dict(\n        xaxis = dict(nticks=4, range=[0,.5],),\n        yaxis = dict(nticks=4, range=[0,.5],),\n        zaxis = dict(nticks=4, range=[0,1000000000],),\n         camera = dict(\n    up=dict(x=0, y=0, z=1),\n    center=dict(x=0, y=0, z=0),\n    eye=dict(x=1.25, y=-1.25, z=1.25)\n    )\n        ),\n    \n    width=700,\n    margin=dict(r=20, l=10, b=10, t=10))\n\n\nfig0.show()\n"

In [ ]:
'''fig1 = go.Figure(fig0)
fig1.update_layout(
  #print(fig)
    scene = dict(
        xaxis = dict(nticks=4, range=[0,.5],),
        yaxis = dict(nticks=4, range=[0,.5],),
        zaxis = dict(nticks=4, range=[0,100000000],),
        camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25*math.cos(math.pi), y=1.25*math.sin(math.pi), z=1.25)
    )
        ),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))
fig0
'''

'fig1 = go.Figure(fig0)\nfig1.update_layout(\n  #print(fig)\n    scene = dict(\n        xaxis = dict(nticks=4, range=[0,.5],),\n        yaxis = dict(nticks=4, range=[0,.5],),\n        zaxis = dict(nticks=4, range=[0,100000000],),\n        camera = dict(\n    up=dict(x=0, y=0, z=1),\n    center=dict(x=0, y=0, z=0),\n    eye=dict(x=1.25*math.cos(math.pi), y=1.25*math.sin(math.pi), z=1.25)\n    )\n        ),\n    width=700,\n    margin=dict(r=20, l=10, b=10, t=10))\nfig0\n'

In [ ]:
fig = go.Figure(frames=frames)

In [ ]:
#Example

In [ ]:
'''import plotly.graph_objects as go
import numpy as np

# Helix equation
t = np.linspace(0, 10, 50)
x, y, z = np.cos(t), np.sin(t), t

fig= go.Figure(go.Scatter3d(x=x, y=y, z=z, mode='markers'))

x_eye = -1.25
y_eye = 2
z_eye = 0.5

fig.update_layout(
         title='Animation Test',
         width=600,
         height=600,
         scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),
         updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=0.8,
                  xanchor='left',
                  yanchor='bottom',
                  pad=dict(t=45, r=10),
                  buttons=[dict(label='Play',
                                 method='animate',
                                 args=[None, dict(frame=dict(duration=60, redraw=True),
                                                             transition=dict(duration=0),
                                                             fromcurrent=True,
                                                             mode='immediate'
                                                            )]
                                            )
                                      ]
                              )
                        ]
)


def rotate_z(x, y, z, theta):
    w = x+1j*y
    return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

frames=[]
for t in np.arange(0, 6.26, 0.1):
    xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
    frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
fig.frames=frames

fig.show()
'''

In [ ]:
'''def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "z": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

fig.update_layout(
         title='Slices in volumetric data',
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()
'''

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Slider: 'z'

Did you mean "x"?

    Valid properties:
        active
            Determines which button (by index starting from 0) is
            considered active.
        activebgcolor
            Sets the background color of the slider grip while
            dragging.
        bgcolor
            Sets the background color of the slider.
        bordercolor
            Sets the color of the border enclosing the slider.
        borderwidth
            Sets the width (in px) of the border enclosing the
            slider.
        currentvalue
            :class:`plotly.graph_objects.layout.slider.Currentvalue
            ` instance or dict with compatible properties
        font
            Sets the font of the slider step labels.
        len
            Sets the length of the slider This measure excludes the
            padding of both ends. That is, the slider's length is
            this length minus 

In [ ]:
#for fig in figs:
  #fig.show()

In [ ]:
 camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1.25, y=-1.25, z=1.25)
    )

fig.update_layout(scene_camera=camera)
fig.show()

 camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.25)
    )

fig.update_layout(scene_camera=camera)
fig.show()

 camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=1.25)
    )

fig.update_layout(scene_camera=camera)
fig.show()

# Estimate

## Example

In [ ]:
triplets = []
prop = .01
MDE = .05
prop2 = prop*(1+MDE)
es = sms.proportion_effectsize(prop, prop2, method='normal')
n = sms.NormalIndPower().solve_power(es, power=0.8, alpha=0.05, ratio=1)
estimate = 10*math.pi*(1/prop)*(1/MDE**2)
error = (2*n - estimate)
error_fraction = error/(2*n)
triplet = ((MDE,prop,error_fraction))
triplets.append(triplet)

prop = .02
MDE = .05
prop2 = prop*(1+MDE)
es = sms.proportion_effectsize(prop, prop2, method='normal')
n = sms.NormalIndPower().solve_power(es, power=0.8, alpha=0.05, ratio=1)
estimate = 10*math.pi*(1/prop)*(1/MDE**2)
error = (2*n - estimate)
error_fraction = error/(2*n)
triplet = ((MDE,prop,error_fraction))
triplets.append(triplet)

triplets

[(0.05, 0.01, 0.013497798485575665), (0.05, 0.02, 0.0031755198574253914)]

In [ ]:
triplets = []
for MDE in MDEs:
  for prop in props:
    prop2 = prop*(1+MDE)
    es = sms.proportion_effectsize(prop, prop2, method='normal')
    n = sms.NormalIndPower().solve_power(es, power=0.8, alpha=0.05, ratio=1)
    estimate = 10*math.pi*(1/prop)*(1/MDE**2)
    error = (2*n - estimate)
    error_fraction = error/(2*n)
    triplet = ((MDE,prop,error_fraction))
    triplets.append(triplet)

In [ ]:
triplets_array = np.array(triplets)
triplets_array.shape

(400, 3)

In [ ]:
t = np.linspace(0, 10, 50)
#x, y, z = np.cos(t), np.sin(t), t

fig= go.Figure(go.Mesh3d(x=triplets_array[:,0], #MDEs
    y=triplets_array[:,1], #props
    #z=triplets_actual_array[:,2], #sample size, y=y, z=z, mode='markers'
    z = triplets_array[:,2],
    intensity=np.log10(triplets_array[:,2])
                         ))

x_eye = -1.25
y_eye = 2
z_eye = 0.5

fig.update_layout(
         title='Sample Size Surface',
         width=600,
         height=600,
    scene=dict(
        xaxis_title='MDE',
        yaxis_title='Incidence',
        zaxis_title='Sample Size',
        zaxis=dict(dtick=1,type='log')),
         scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),

         updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=0.8,
                  xanchor='left',
                  yanchor='bottom',
                  pad=dict(t=45, r=10),
                  buttons=[dict(label='Play',
                                 method='animate',
                                 args=[None, dict(frame=dict(duration=60, redraw=True),
                                                             transition=dict(duration=0),
                                                             fromcurrent=True,
                                                             mode='immediate'
                                                            )]
                                            )
                                      ]
                              )
                        ]
)


def rotate_z(x, y, z, theta):
    w = x+1j*y
    return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

frames=[]
for t in np.arange(0, 6.28, 0.1):
    xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
    frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
fig.frames=frames

fig.show()

<ipython-input-17-8d9a90079f39>:8: RuntimeWarning:

invalid value encountered in log10



In [ ]:
MDEs.size, props.size, triplets_array.size

(20, 20, 0)

In [ ]:
mask1 = triplets_actual_array[:,0]>.1
mask2 = triplets_actual_array[:,1]>.1
mask3 = np.logical_and(mask1,mask2)
triplets_actual_array[mask3][:,2].max()

12303.422421410272

In [ ]:
props.max(), triplets_array[:,2].min(), triplets_array[:,2].max()

(0.2, -0.24615721486170727, 0.1811404779933956)

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
#z = z_data
#x, y = MDEs, props
fig = go.Figure(data=[go.Mesh3d(
    x=triplets_array[:,0],
    y=triplets_array[:,1],
    z=triplets_array[:,2],
  opacity=0.5,
  color='rgba(244,22,100,0.6)'
)])
fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[0,.5],),
        yaxis = dict(nticks=4, range=[0,.5],),
        zaxis = dict(nticks=4, range=[-.25,.25],),
        ),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()

In [ ]:
 fig = go.Figure(data=[go.Mesh3d(
    x=triplets_array[:,0],
    y=triplets_array[:,1],
    z=triplets_array[:,2],
            intensity=z,
  #opacity=0.5,
  color='rgba(0,0,255,0.6)'
)])
 camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1.25, y=-1.25, z=1.25)
    )

fig.update_layout(scene_camera=camera)
fig.show()


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
 fig = go.Figure(data=[go.Mesh3d(
    x=triplets_actual_array[:,0],
    y=triplets_actual_array[:,1],
    z=triplets_actual_array[:,2],
    intensity=z,
  #opacity=0.5,
  color='rgba(0,0,255,0.6)'
)])
 camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.25)
    )

fig.update_layout(scene_camera=camera)
fig.show()

NameError: name 'z' is not defined

In [ ]:
x.size

20

In [ ]:
((55*np.random.randn(70))).size

70

In [ ]:
fig

In [ ]:
import plotly.graph_objects as go
import numpy as np
np.random.seed(1)

N = 70

fig = go.Figure(data=[go.Mesh3d(x=(70*np.random.randn(N)),
                   y=(55*np.random.randn(N)),
                   z=(40*np.random.randn(N)),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[-100,100],),
                     yaxis = dict(nticks=4, range=[-50,100],),
                     zaxis = dict(nticks=4, range=[-100,100],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()

In [ ]:
triplets_array = np.array(triplets)

In [ ]:
triplets_array

array([[ 0.01      ,  0.01      , -0.00578839],
       [ 0.01      ,  0.01191919, -0.00775185],
       [ 0.01      ,  0.01383838, -0.009723  ],
       ...,
       [ 0.5       ,  0.19616162, -0.07019112],
       [ 0.5       ,  0.19808081, -0.07360257],
       [ 0.5       ,  0.2       , -0.07703624]])

In [ ]:
np.shape(triplets_array)

(10000, 3)

In [ ]:
triplets_array[:,2]

array([-0.00578839, -0.00775185, -0.009723  , ..., -0.07019112,
       -0.07360257, -0.07703624])

In [ ]:
np.max((abs(triplets_array[:,2])))

0.24615721486170727

In [ ]:
np.argmax(abs(triplets_array[:,2]))

99

In [ ]:
triplets_array[49]

array([ 0.01      ,  0.1040404 , -0.11194556])